# Quickstart

`Bionty` focuses on improving a computational biologist's experience when it comes to bio entities. Our goal is to build something that's interpretable, intuitive and easy to use. 

In this notebook, we'll introduce the API that defines entities species, genes, proteins and cell types.

It shows how to 

- query different semantic representations based on underlying ontologies
- standardize columns in `DataFrame` objects

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import bionty as bt
import pandas as pd

## Species

Let's start with exploring the `Species`! You can initiate the `Species` class by providing a common name of species.

Don't worry if you are not sure what common name to use, just go with default for now.

In [ ]:
species = bt.Species(common_name="human")

Now you have access to a couple of attributes, let's see what they are.

- `.fields`: this gives you a list of column names that annotate each entry, in this case each species
- `.std_id`: this prints the field which is used as the standardized id. Note that Bionty focuses on interpretability, so we use the unique `common_name` of [Ensembl Species](https://asia.ensembl.org/info/about/species.html).
- `.std_name`: this is the value of `std_id` in the current instance. (e.g. `human`, `mouse`, `dog` ...)

In [ ]:
print(
    f"`.fields:` {species.fields}\n`.std_id`: {species.std_id}\n`.std_name`:"
    f" {species.std_name}"
)

For entities that require name conversions, we implement a `.search` function to allow converting among different reference ids.

In [ ]:
print(
    f'`.scientific_name`: {species.search("scientific_name")}\n`.taxon_id`:'
    f' {species.search("taxon_id")}'
)

A key feature of `Bionty` is that, for each entity class, we implement a static `.dataclass` via `pydantic` models which allows users to access each entry via tab completion. This allows you quickly search for a entry of interest and retrieve it's associated attributes.

In [ ]:
dc = species.dataclass

In [ ]:
dc.cat

In [ ]:
dc.cat.scientific_name

## Gene

Next let's take a look at genes, which essentially follows the same design choices as `Species`.

The main differences are:
1. The `.dataclass` of `Gene` is species specific, therefore you will only retrieve gene entries of the specified species.
2. We implement a `.standardize` function which allows to standardizing gene names inplace in a dataframe.

In [ ]:
gene = bt.Gene(species="human")

In [ ]:
dc = gene.dataclass

In [ ]:
dc.PDCD1

In [ ]:
dc.PDCD1.ensembl_gene_id

Same as species, you can check out gene related fields

In [ ]:
print(f"`.fields:` {gene.fields}\n`.std_id`: {gene.std_id}")

Now Let's check out a few examples of gene name conversions.

The `.search` function by default converts to the standardized ids, you can also specify `.id_type_from` and `id_type_to` to control the behavior.


In [ ]:
hgnc_ids = ["HGNC:1100", "HGNC:1101"]
ensembl_ids = ["ENSG00000012048", "ENSG00000139618"]

In [ ]:
# default is to convert into .std_id

gene.search(ensembl_ids, id_type_from="ensembl.gene_id")

In [ ]:
# OR you can convert between any two of the attributes

gene.search(["BRCA1", "BRCA2"], id_type_from="hgnc_symbol", id_type_to="entrez.gene_id")

Now let's try to provide a dataframe.

`.standardize` produces:
- The outcome of standardization will be directly written as the index.
- A `std_id` column containing the standardized ids, if no standardized id is found for a gene, it will be NaN.
- A `index_orig` column containing the original index provided before standardization.

In [ ]:
# default is to standardizing gene symbols

df = pd.DataFrame(index=["RNF53", "BRCA2", "FakeGene"])
gene.standardize(df)

df

Of course you can also just provide a list of genes without a dataframe!

In [ ]:
gene.standardize(["RNF53", "BRCA2", "FakeGene"])

Let's now try standardizing some ensembl ids

In [ ]:
gene.standardize(["ENSG00000012048", "ENSG00000139618"], id_type="ensembl.gene_id")

## Protein

Protein is very similar to Gene.

In [ ]:
protein = bt.Protein(species="human")

In [ ]:
print(f"`.fields:` {protein.fields}\n`.std_id`: {protein.std_id}")

In [ ]:
uniprot_ids = ["P40925", "P40926", "O43175", "Q9UM73"]

protein.search(uniprot_ids, id_type_from="UNIPROT_ID", id_type_to="CHEMBL_ID")

## Cell Type

(More features to come!)

Here we provide an interface to the Cell Ontology via an ontology manager Owlready2.

Other ontology based entities such as `Disease` and `Tissue` are both implemented this way.

- `.onto` is the ontology instance of Owlready2, which allows you to load, query, modify, save ontologies. See more [here](https://owlready2.readthedocs.io/en/latest/intro.html).
- `.onto_dict` is the dictionary of {'name': 'label'}.
- `.classes` is the access point to each ontology object.
- `.search` allows you to look for ontologies based on keywords.
- `.standardize` checks the correctness of ontology ids.
- `.dataclass` is a static dataclass for accessing entities with tab completion.


In [ ]:
ct = bt.CellType()

In [ ]:
ct.onto

In [ ]:
ct.onto_dict["CL_0002000"]

A few info available in an object 

In [ ]:
obj = ct.classes["CL_0002000"]

In [ ]:
obj.is_a

In [ ]:
obj.ancestors()

Let's try to search for some CL terms

In [ ]:
ct.search(["T cell", "B cell", "hepatocyte"])

Standardization currently merely prints out warnings of obsolete or notfound ontologies. 

In [ ]:
terms = ["CL:0000084", "CL:0000243", "CL:0002000"]

ct.standardize(terms)

Similarly, it has a static dataclass for accessing each entity

In [ ]:
dc = ct.dataclass

In [ ]:
dc.CL_0000738